In [7]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from pathlib import Path
from scipy.ndimage import binary_dilation


np.set_printoptions(precision=4)


In [10]:
def fill_cracks_and_disocclusions(virtual_image):
        """
        Fill cracks and disocclusions in the synthesized virtual image.

        Parameters:
        virtual_image (numpy.ndarray): The virtual view image with artifacts.

        Returns:
        filled_image (numpy.ndarray): The virtual image with cracks and disocclusions filled.
        """
        virtual_image_colors = virtual_image.copy()

        #filled_image=virtual_image.copy()
        gray_image = cv2.cvtColor(virtual_image, cv2.COLOR_BGR2GRAY)

        # Identify empty pixels (where the virtual image has no texture information)
        empty_mask = np.isnan(gray_image) | (gray_image == 0)

        # Convert the mask to a single-channel 8-bit image
        empty_mask_uint8 = empty_mask.astype(np.uint8)
        # Use inpainting to fill empty pixels
        filled_image = cv2.inpaint(gray_image.astype(np.uint8), empty_mask_uint8,
                                  inpaintRadius=3, flags=cv2.INPAINT_TELEA)

        print("sarah")
        combined_image = cv2.addWeighted(virtual_image_colors, 0.7, filled_image, 0.3, 0)
        filled_image_bgr = cv2.cvtColor(combined_image, cv2.COLOR_GRAY2BGR)

        return filled_image_bgr

def load_image(image_path, depth_path):
        """
        Load 2D original image (V) and a corresponding depth map (D).

        Parameters:
        image_path (str): Image path.
        depth_path (str): Depth image path.

        Returns:
        numpy.ndarray: Image V.
        numpy.ndarray: Depth image D.
        """
        try:
            V = cv2.imread(image_path)
            D = cv2.imread(depth_path, cv2.IMREAD_GRAYSCALE)
        except IOError:
            print("Image file is corrupted.")
        return V, D

def depth_map_to_distance_map(D, Z_near, Z_far):
      """
        Transform the integerbased depth map D value ranges into
        the floating point geometrical distance map Z

        Parameters:
        D (numpy.ndarray): depth map.
        Z_near (float): depth map normalization factor .
        Z_far (float): depth map normalization factor.

        Returns:
        numpy.ndarray: the floating point geometrical distance map Z.
      """
      # based on ---- Habigt, Julian Albert. Hole-Filling Algorithms for Depth-Image-Based Rendering.
      # Diss. Technische Universität München, 2020.
      D=D.astype(np.float64)
      tmp=((D/ 255.0)*((1/Z_near) - (1/Z_far))) +(1/Z_far)
      Z = 1/tmp
      return Z
def plot_images_in_list(img_list, rows, cols):
    """
    Plots images from a list in a grid of subplots.

    Parameters:
    img_list (list[np.ndarray]): List of images.
    rows (int):  Number of rows in the subplot grid.
    cols (int): Number of columns in the subplot grid.
    """

    total_slots = rows * cols
    num_images = len(img_list)

    # Create a figure with the specified number of subplots
    fig, axes = plt.subplots(rows, cols, figsize=(15, 10),constrained_layout=True)
    # If there's only one subplot, axes is not an array but a single object
    if total_slots == 1:
        axes = [axes]  # Convert to a list for consistency
    # If there's more than one subplot, flatten the axes array for easy iteration
    elif total_slots > 1:
        axes = axes.flatten()

    # Loop through the list of images and corresponding axes
    for ax, img in zip(axes, img_list):
            image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            ax.imshow(image_rgb)
            ax.axis('off')  # Turn off axis labels
    plt.show()


def save_images(imageList,directory):
    """
    saves imagelist in directory with virtual_view_{i}.png name.

    Parameters:
    imageList (List[np.ndarray]): image list.
    directory (str): location for saving images.
    """
    path = Path(directory)
    path.mkdir(parents=True, exist_ok=True)
    for i, view in enumerate(imageList):
      filename= f"virtual_view_{i}.png"
      path = os.path.join(directory, filename)
      cv2.imwrite(path, view)
    print(f"{len(imageList)} Images succesfully saved in: {directory}.")

class DIBR:
  """
  A class to Depth Image Based Rendering.

  Attributes
  ----------
  V_o (numpy.ndarray): Original 2D image.
  D_o (numpy.ndarray): Depth map corresponding to the image.
  K_o (numpy.ndarray): Intrinsic matrix of the original camera.
  Rt_o (numpy.ndarray): Rotation matrix|Translation vector of the original camera.
  K_v (numpy.ndarray): Intrinsic matrix of the virtual camera.
  Rt_v (numpy.ndarray): Rotation matrix|Translation vector of the virtual camera.
  p_original(numpy.ndarray): original projection matrice
  p_virtual(numpy.ndarray): virtual projection matrice


  Methods
  -------
  compute_projection_matrix(k,Rt):
    returns projection matrice
  render(self):
    returns synthesized virtual .
  show_image(image)
    returns nothing and shows image
  save_images(imageList,directory):
    returns nothing and saves images in imageList in directory
  render_virtual_views(N):
    renders N image that evenly spread out on the line connecting the original and virtual camera centers
  """

  def  __init__(self, V_o, Z_o, K_o, Rt_o, K_v, Rt_v):
    """
    Initialize the Depth_Image_Based_Rendering with the original image, depth map, and camera parameters.

    Parameters:
    V_o (numpy.ndarray): Original 2D image.
    D_o (numpy.ndarray): Depth map corresponding to the image.
    K_o (numpy.ndarray): Intrinsic matrix of the original camera.
    Rt_o (numpy.ndarray): Rotation matrix|Translation vector of the original camera.
    K_v (numpy.ndarray): Intrinsic matrix of the virtual camera.
    Rt_v (numpy.ndarray): Rotation matrix|Translation vector of the virtual camera.
    """
    self.V_o = V_o
    self.Z_o = Z_o
    self.K_o = K_o
    self.Rt_o = Rt_o
    self.K_v = K_v
    self.Rt_v = Rt_v

    # Compute projection matrices
    self.p_original=self.compute_projection_matrix(self.K_o,self.Rt_o)
    self.p_virtual=self.compute_projection_matrix(self.K_v,self.Rt_v)


  def compute_projection_matrix(self,k,Rt):
    """
    Calculate Projection Matrix with k and Rt of camera.

    Parameters:
    K (numpy.ndarray): Camera intrinsic matrix (3x3).
    Rt (numpy.ndarray): Camera rotation|translation vector matrix (3x4).

    Returns:
    numpy.ndarray: projection matrix (3x4).
    """
    p=k@Rt
    return p


  def render_lowPerf(self):
    """
    synthesize virtual image according 3 stepes .
    1.reconstruct scene point M
    2.render virtual image position m
    3.replace pixel in the target position with source image pixels value

    Parameters:
    self.V_o(numpy.ndarray):original image
    self.D_o(numpy.ndarray):original image depth
    self.K_o(numpy.ndarray):Original Camera intrinsic matrix
    self.Rt_o(numpy.ndarray):Original Camera rotation|translation vector matrix
    self.K_v(numpy.ndarray):virtual Camera intrinsic matrix
    self.Rt_v(numpy.ndarray):virtual Camera rotation|translation vector matrix

    Returns:
    numpy.ndarray: virtual image.
    """

    # Iterate through each pixel in the original image
    # implemented from recieved document

    # Get original image dimensions
    height, width = self.V_o.shape[:2]
    # Initialize empty virtual image
    V_virtual = np.zeros_like(self.V_o)
    for y in range(height):
        for x in range(width):
            # Depth value
            depth_value = self.Z_o[y, x]
            if depth_value <= 0:
                continue  # Skip if the depth value is not valid

            # 1. Reconstruct 3D world coordinates in M from image m
            m_org_coords = np.array([x, y, 1]).transpose()
            M_coords =np.linalg.pinv(self.p_original) @ (depth_value * m_org_coords)
            # M homogeneous
            M_coords[3] = 1

            # 2. Project the 3D point from M point to the m in image plane
            m = self.p_virtual @ M_coords.transpose()
            x_y=m[:2]
            m_homogeneous = x_y/depth_value

            # 3.  transfer value or color in position of specific pixel
            # in the original image to  virtual image color
            x_virtual, y_virtual = int(m_homogeneous[0]), int(m_homogeneous[1])
            if 0 <= x_virtual < width and 0 <= y_virtual < height:
                V_virtual[y_virtual, x_virtual] = self.V_o[y, x]

    return V_virtual

  def render_virtual_views(self,N):
    """
    rendering N virtual Views.

    Parameters:
    N (int): The number of views.

    Returns:
    List[np.ndarray]: list of N views.
    """
    virtual_views = []
    # original camera position
    t_o = self.Rt_o[:, 3]
    # virtual camera position
    t_v = self.Rt_v[:, 3]
    R_o=self.Rt_o[:, :3]

    virtual_views = [None] * N
    for i in range(N):
        # Linear interpolation is used to find a point along a straight line between 0,1.
        # alpha controls the interpolation
        # Computer Graphics: Principles and Practice by John F. Hughes,
        # Andries van Dam, Morgan McGuire, David F. Sklar, James D. Foley, Steven K. Feiner, Kurt Akeley. Addison-Wesley, 2013
        alpha = i / (N - 1)
        # t_current smoothly transitions from t_o to t_v.
        t_current = (1 - alpha) * t_o + alpha * t_v
        Rt_current = np.hstack((R_o, t_current[:, None]))
        self.Rt_v=Rt_current
        print("Rt_current:",Rt_current)
        V_v = self.render()
        virtual_views[i] = V_v
        print(f"The view number{i+1} is rendered.")

    return virtual_views

  def render(self):
      """
      Synthesize a virtual image according to the following steps:
      1. Reconstruct scene point M in 3D.
      2. Render virtual image position m by projecting M.
      3. Replace the pixel in the target position with the source image pixel value.

      Parameters:
      self.V_o (numpy.ndarray): Original image.
      self.D_o (numpy.ndarray): Original image depth.
      self.K_o (numpy.ndarray): Original Camera intrinsic matrix.
      self.Rt_o (numpy.ndarray): Original Camera rotation|translation matrix.
      self.K_v (numpy.ndarray): Virtual Camera intrinsic matrix.
      self.Rt_v (numpy.ndarray): Virtual Camera rotation|translation matrix.

      Returns:
      numpy.ndarray: Virtual image.
      """

      # Get original image dimensions
      height, width = self.V_o.shape[:2]

      # Initialize empty virtual image
      V_virtual = np.zeros_like(self.V_o)

      # Depth values (flattened)
      depth_values = self.Z_o.flatten()
      # Filter out invalid depth values
      valid_depth = depth_values > 0
      depth_values = depth_values[valid_depth]


      # Generate a grid of coordinates
      x, y = np.meshgrid(np.arange(width), np.arange(height))
      ones = np.ones_like(x)
      # Stack into homogeneous coordinates (3D)
      m_org_coords = np.stack((x, y, ones), axis=-1).reshape(-1, 3).T
      # Filter out invalid coordinates
      m_org_coords = m_org_coords[:, valid_depth]


      # 1. Reconstruct 3D world coordinates in M from image m
      M_coords = np.linalg.pinv(self.p_original) @ (depth_values * m_org_coords)
      # Ensure M_coords is homogeneous
      M_coords = np.vstack((M_coords[:3], np.ones_like(depth_values)))

      # 2. Project the 3D point M to the m in the image plane of the virtual camera
      m_virtual = self.p_virtual @ M_coords
      m_virtual = (m_virtual[:2] / m_virtual[2]).astype(int)

      # 3. Transfer the pixel value from the original image to the virtual image
      x_virtual, y_virtual = m_virtual
      valid_indices = (0 <= x_virtual) & (x_virtual < width) & (0 <= y_virtual) & (y_virtual < height)

      V_virtual[y_virtual[valid_indices], x_virtual[valid_indices]] = self.V_o.reshape(-1, 3)[valid_depth][valid_indices]

      return V_virtual